In [1]:
import pandas as pd
import numpy as np
import scipy
import librosa

In [11]:
# lese db_snapshot ein
df = pd.read_csv("data/db_data/db_snapshot.csv")
df.head()

,Unnamed: 0,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de,confidence,min_conf,current_location,active_status
0,0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke,0.911344,0.85,0101000020E6100000000000A0F7A81F40000000C0FA2A...,False
1,1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke,0.907262,0.85,0101000020E6100000000000A0F7A81F40000000C0FA2A...,False
2,2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke,0.920276,0.85,0101000020E6100000000000A0F7A81F40000000C0FA2A...,False
3,3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke,0.976985,0.85,0101000020E6100000000000A0B2A81F4000000060F72A...,True
4,4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke,0.984349,0.85,0101000020E6100000000000A0B2A81F4000000060F72A...,True


In [12]:
# remove unneeded columns
df.drop(['Unnamed: 0', 'current_location', 'active_status', 'confidence', 'min_conf'], axis=1, inplace=True)
print(df.shape)
df.head()

(129, 7)


,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke
1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke
2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke
4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke


In [13]:
# filter out duplicates
df = df.drop_duplicates()
print(df.shape)
df.head()

(128, 7)


,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke
1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke
2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke
4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke


In [20]:
# create dataframes based on species as we do not compare different species, only within the species

df_dorngras = df[df['species_name_de'].str.contains('Dorngrasmücke')]
df_trauer = df[df['species_name_de'].str.contains('Trauerschnäpper')]
df_grau = df[df['species_name_de'].str.contains('Grauschnäpper')]
df_gold = df[df['species_name_de'].str.contains('Goldammer')]
df_blut = df[df['species_name_de'].str.contains('Bluthänfling')]


In [22]:
print(f"Dorngrasmücke: {df_dorngras.shape}")
print(f"Trauerschnäpper: {df_trauer.shape}")
print(f"Grauschnäpper: {df_grau.shape}")
print(f"Goldammer: {df_gold.shape}")
print(f"Bluthänfling: {df_blut.shape}")

Dorngrasmücke: (14, 7)
Trauerschnäpper: (19, 7)
Grauschnäpper: (77, 7)
Goldammer: (6, 7)
Bluthänfling: (12, 7)


In [23]:
df_dorngras.head(20)
# problem: noted timestamp of playback does not match any timestamp in dataframe

,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke
1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke
2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke
4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke
5,2022-02-23 10:16:30.000177,0 days 00:00:14.165333,6030536,6030536_1645611390-177.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
6,2022-02-23 10:16:30.000177,0 days 00:00:14.165333,6030536,6030536_1645611390-177.flac,0 days 00:00:03,0 days 00:00:06,Dorngrasmücke
7,2022-02-23 10:16:30.000177,0 days 00:00:14.165333,6030536,6030536_1645611390-177.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke
8,2022-02-23 10:16:14.984882,0 days 00:00:14.165333,6030536,6030536_1645611374-984882.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
9,2022-02-23 10:16:14.984882,0 days 00:00:14.165333,6030536,6030536_1645611374-984882.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke


In [ ]:
# put same call detected by multiple boards in one group
